# Bring in geography

It is time to get your hands on some spatial data. You will not go far
from your `pandas` experience, you’ll just expand it a bit. This section
covers an introduction to `geopandas`, a Python package extending the
capabilities of `pandas` by providing support for geometries,
projections and geospatial file formats. Let’s start with importing
`geopandas`.

In [2]:
import geopandas as gpd

## Datasets

You will be using a few different datasets in this notebook. The first
one contains data on buildings, streets and street junctions of a small
part of Paris from @fleischmann2021Evolution. The data contain some
information on urban morphology derived from these geometries, but
today, you will be mostly interested in geometries, not so much in
attributes.

## Reading files

Assuming you have a file containing both data and geometry
(e.g. GeoPackage, GeoJSON, Shapefile), you can read it using
`geopandas.read_file()`, which automatically detects the file type and
creates a `geopandas.GeoDataFrame`. A `geopandas.GeoDataFrame` is just
like `pandas.DataFrame` but with additional column(s) containing
geometry.

In [ ]:
paris_url = (
    "https://github.com/martinfleis/evolution-gean/raw/main/data/Paris.gpkg"
)
buildings = gpd.read_file(paris_url, layer="buildings")
buildings.head()

> **Explore available layers**
>
> You can quickly check which layers are available in the GPKG file
> using `geopandas.list_layers()`.
>
> ``` python
> gpd.list_layers(paris_url)
> ```
>
> <div>
> <style scoped>
>     .dataframe tbody tr th:only-of-type {
>         vertical-align: middle;
>     }
>
>     .dataframe tbody tr th {
>         vertical-align: top;
>     }
>
>     .dataframe thead th {
>         text-align: right;
>     }
> </style>
>
> |     | name         | geometry_type |
> |-----|--------------|---------------|
> | 0   | tessellation | Unknown       |
> | 1   | buildings    | Polygon       |
> | 2   | edges        | LineString    |
> | 3   | nodes        | Point         |
>
> </div>

Let’s have a quick look at the `"geometry"` column. This is the special
one enabled by `geopandas`. You can notice that the objects stored in
this column are not `float` or `string` but Polygons of a `geometry`
data type instead. The column is also a `geopandas.GeoSeries` instead of
a `pandas.Series`.

In [ ]:
buildings["geometry"].head()

Polygons are not the only geometry types you can work with. The same
GPKG that contains buildings data also includes street network
geometries of a LineString geometry type.

In [ ]:
street_edges = gpd.read_file(paris_url, layer="edges")
street_edges.head(2)

You can also load another layer, this time with Point geometries
representing street network junctions.

In [ ]:
street_nodes = gpd.read_file(paris_url, layer="nodes")
street_nodes.head(2)

## Writing files

To write a `GeoDataFrame` back to a file, use `GeoDataFrame.to_file()`.
The file is format automatically inferred from the suffix, but you can
specify your own with the `driver=` keyword. When no suffix is given,
GeoPandas expects that you want to create a folder with an ESRI
Shapefile.

In [7]:
buildings.to_file("buildings.geojson")

## Geometries

Geometries within the *geometry* column are `shapely` objects. GeoPandas
itself is not creating the object but leverages the existing ecosystem
(note that there is a significant overlap of the team writing both
packages to ensure synergy). A typical `GeoDataFrame` contains a single
geometry column, as you know from traditional GIS software. If you read
it from a file, it will most likely be called `"geometry"`, but that is
not always the case. Furthermore, a `GeoDataFrame` can contain multiple
geometry columns (e.g., one with polygons, another with their centroids
and another with bounding boxes), of which one is considered *active*.
You can always get this active column, no matter its name, by using
`.geometry` property.

In [ ]:
buildings.geometry.head()

> **Property vs indexing**
>
> In data frames, you can usually access a column via indexer
> (`df["column_name"]`) or a property (`df.column_name`). However, the
> property is not available when there is either a method (e.g. `.plot`)
> or a built-in property (e.g. `.crs` or `.geometry`) overriding this
> option.

You can quickly check that the geometry is a data type indeed coming
from `shapely`. You will use `shapely` directly in some occasions but in
most cases, any interaction with `shapely` will be handled by
`geopandas`.

In [ ]:
type(buildings.geometry.loc[0])

There is also a handy SVG representation if you are in a Jupyter
Notebook.

In [ ]:
buildings.geometry.loc[0]

If you’d rather see a text representation, you can retrieve a Well-Known
Text using `.wkt` property.

In [ ]:
buildings.geometry.loc[0].wkt

## Projections

But without an assigned coordinate reference system (CRS), you don’t
know where this shape lies on Earth. Therefore, each geometry column has
(optionally) assigned a CRS. That is always available via `.crs`.

In [ ]:
buildings.crs

If you check the type, you’ll notice it comes from the `pyproj` package.
Note that you will likely never interact with that yourself.

In [ ]:
type(buildings.crs)

The object with the CRS information has plenty of useful tricks. You
can, for example, quickly check if it is a geographic (coordinates are
latitude and longitude in degrees) or a projected CRS (x and y in
meters, feet or similar).

In [ ]:
buildings.crs.is_geographic

`geopandas` is like a glue that brings different pieces of the Python
ecosystem together. `pandas` for tabular data, `shapely` for geometries,
`pyogrio` for interaction with GIS file formats or `pyproj` for CRS
management.

## Simple accessors and methods

Now you have a `GeoDataFrame` and can start working with its geometry.

Since there was only one geometry column in the buildings dataset, this
column automatically becomes the *active* geometry and spatial methods
used on the `GeoDataFrame` will be applied to the `"geometry"` column.

### Measuring area

To measure the area of each polygon, access the `GeoDataFrame.area`
attribute, which returns a `pandas.Series`. Note that
`GeoDataFrame.area` is just `GeoSeries.area` applied to the *active*
geometry column.

In [ ]:
buildings["area"] = buildings.area
buildings["area"].head()

### Getting polygon boundary and centroid

`geopandas` allows you a quick manipulation of geometries. For example,
to get the boundary of each polygon (of a LineString geometry type),
access the `GeoDataFrame.boundary` property:

In [ ]:
buildings["boundary"] = buildings.boundary
buildings["boundary"].head()

Since you have saved the boundary as a new column, you now have two
geometry columns in the same `GeoDataFrame`.

You can also create new geometries, which could be, for example, a
buffered version of the original one (i.e., `GeoDataFrame.buffer(10)` to
buffer by 10 meters if your CRS is in meters) or its centroid:

In [ ]:
buildings["centroid"] = buildings.centroid
buildings["centroid"].head()

### Measuring distance

Measuring distance is similarly straightforward. The building data are
from central Paris, so you can try to figure out how far is each of them
from the [Arc de
Triomphe](https://en.wikipedia.org/wiki/Arc_de_Triomphe).

Use the coordinates of the Arc de Triomphe to generate a Point geometry.

In [18]:
arc = gpd.GeoSeries.from_xy(x=[2.29503], y=[48.87377], crs="EPSG:4326")

> **Use geocoding to get the geometry**
>
> The code above uses known coordinates. If you don’t know them but know
> the address and a name of a place, you can use the built-in geocoding
> capabilities in `geopandas`:
>
> ``` py
> arc = gpd.tools.geocode("Arc de Triomphe, Paris")
> ```

Now you have the Arc de Triomphe as a Point. However, that point is in
latitude and longitude coordinates, which is a different CRS than the
one `buildings` use. They must use the same CRS to measure the distance
between geometries.

In [ ]:
arc = arc.to_crs(buildings.crs)
arc

With a Point based on the correct CRS, you can measure the distance from
each building to the Arc.

In [ ]:
arc_location = arc.geometry.item()
buildings["distance_to_arc"] = buildings.distance(arc_location)
buildings["distance_to_arc"].head()

Using `buildings.distance(arc_location)` measures the distance from
geometries in the active geometry column, which are Polygons in this
case. But you can also measure distance from geometries in any other
column.

In [ ]:
buildings["centroid"].distance(arc_location).head()

Note that `geopandas.GeoDataFrame` is a subclass of `pandas.DataFrame`,
so you have all the `pandas` functionality available to use on the
geospatial dataset — you can even perform data manipulations with the
attributes and geometry information together.

For example, to calculate the average of the distances measured above,
access the `"distance"` column and call the `.mean()` method on it:

In [ ]:
buildings["distance_to_arc"].mean()

Similarly, you can plot the distribution of distances as a histogram.

In [ ]:
_ = buildings["distance_to_arc"].plot.hist(bins=50)

## Making maps

Maps in GeoPandas are of two kinds. Static images and interactive maps
based on [leaflet.js](http://leafletjs.com).

### Static maps

GeoPandas can also plot maps to check how the geometries appear in
space. Call `GeoDataFrame.plot()` to plot the active geometry. To colour
code by another column, pass in that column as the first argument. In
the example below, you can plot the active geometry column and colour
code by the `"distance_to_arc"` column. You may also want to show a
legend (`legend=True`).

In [ ]:
_ = buildings.plot("distance_to_arc", legend=True)

The map is created using the `matplotlib` package. It’s the same that
was used under the hood for all the plots you have done before. You can
use it directly to save the resulting plot to a PNG file.

In [25]:
import matplotlib.pyplot as plt

If you now create the plot and use the `plt.savefig()` function in the
same cell, a PNG will appear on your disk.

``` py
buildings.plot("distance_to_arc", legend=True)
plt.savefig("distance_to_arc.png", dpi=150)
```

> **Other resources for static plotting**
>
> Want to know more about static plots? Check [this
> chapter](https://darribas.org/gds_course/content/bC/lab_C.html#styling-plots)
> of *A Course on Geographic Data Science* by @darribas_gds_course or
> [the GeoPandas
> documentation](https://geopandas.org/en/stable/docs/user_guide/mapping.html).

### Interactive maps

You can also explore your data interactively using
`GeoDataFrame.explore()`, which behaves in the same way `.plot()` does
but returns an interactive HTML map instead.

In [1]:
buildings.explore("distance_to_arc", legend=True)

NameError: name 'buildings' is not defined

Using the `GeoSeries` of centroids, you can create a similar plot, but
since you access only a single column, it has no values to show.

In [ ]:
buildings["centroid"].explore()

> **Keeping data around**
>
> If you want to use centroids on a map but keep the data around to have
> them available in the tooltip, you can assign it as an active geometry
> and then use `.explore()`.
>
> ``` python
> buildings.set_geometry("centroid").explore()
> ```

You can also layer multiple geometry layers on top of each other. You
just need to use one plot as a map (`m`) object for the others.

In [ ]:
m = buildings.explore(tiles="CartoDB Positron", popup=True, tooltip=False)
street_edges.explore(m=m, color="black")
street_nodes.explore(m=m, color="pink")
arc.explore(m=m, marker_type="marker")
m.save("paris-map.html")
m

## Spatial predicates

Spatial predicates tell you the spatial relation between two geometries.
Are they equal, intersect, overlap, or are they within another?

Let’s leave Paris and load a map of countries provided by
@runfola2020geoboundaries and simplified for this course.

In [29]:
world_countries = gpd.read_file(
    "https://martinfleischmann.net/sds/geographic_data/data/geoboundaries_cgaz.gpkg",
)

> **Alternative**
>
> Instead of reading the file directly off the web, it is possible to
> download it manually, store it on your computer, and read it locally.
> To do that, you can follow these steps:
>
> 1.  Download the file by right-clicking on [this
>     link](https://martinfleischmann.net/sds/geographic_data/data/geoboundaries_cgaz.gpkg)
>     and saving the file
> 2.  Place the file in the same folder as the notebook where you intend
>     to read it
> 3.  Replace the code in the cell above with:
>
> ``` python
> world_countries = gpd.read_file("geoboundaries_cgaz.gpkg")
> ```

Since you’ll be exploring spatial predicates, you need a second layer.
Let’s load a data set of populated places from [Natural
Earth](https://www.naturalearthdata.com).

In [30]:
world_cities = gpd.read_file(
    "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_populated_places_simple.zip"
)

A quick map with both shows how the data look and relate.

In [ ]:
ax = world_countries.plot()
_ = world_cities.plot(ax=ax, color="pink", alpha=0.7)

Let’s first create some small toy spatial objects. First, extract a
polygon representing Belgium.

In [ ]:
belgium = world_countries.loc[
    world_countries["shapeGroup"] == "BEL", "geometry"
].item()
belgium

Second, get points representing Paris and Brussels.

In [33]:
paris = world_cities.loc[world_cities["name"] == "Paris", "geometry"].item()
brussels = world_cities.loc[world_cities["name"] == "Brussels", "geometry"].item()

And create a line connecting both cities. Here comes one of those cases
when you use `shapely` directly.

In [34]:
import shapely

line = shapely.LineString([paris, brussels])

Let’s visualise those four geometry objects together. To do that, you
can create a single `GeoSeries` with all of them. Notice that such a
`GeoSeries` contains mixed geometry types (Polygon, two Points,
LineString). That may be an issue with some traditional GIS software,
but is no problem with `geopandas`.

In [ ]:
gpd.GeoSeries([belgium, paris, brussels, line], crs=world_cities.crs).explore(
    marker_type="marker"
)

You can recognise the shape of Belgium, two markers locating Paris and
Brussels and the line connecting them.

Brussels, the capital of Belgium, is thus located within Belgium. This
is a spatial relationship, and we can test this using the individual
shapely geometry objects as follow:

In [ ]:
brussels.within(belgium)

And using the reverse, Belgium contains Brussels:

In [ ]:
belgium.contains(brussels)

On the other hand, Belgium does not contain Paris:

In [ ]:
belgium.contains(paris)

Nor Paris is located in Belgium:

In [ ]:
paris.within(belgium)

The straight line you draw from Paris to Brussels is not fully located
within Belgium.

In [ ]:
belgium.contains(line)

But it does intersect with it.

In [ ]:
line.intersects(belgium)

### Spatial relationships with GeoDataFrames

The same methods available on individual `shapely` geometries are also
available as methods on `GeoSeries` and `GeoDataFrame` objects.

For example, if you call the `.contains()` method on the
`world_countries` dataset with the `paris` point, it will do this
spatial check for each country in the `world_countries` `GeoDataFrame`.

In [ ]:
world_countries.contains(paris).sum()

Because the above gives us a boolean result, we can use that to filter
the dataframe:

In [ ]:
world_countries[world_countries.contains(paris)].explore()

> **Making use of spatial index**
>
> You could also do the same based on a query over the spatial index.
> Custom queries on a spatial index using `GeoDataFrame.sindex.query()`
> are often much faster but are also considered advanced usage. Since
> GeoPandas wraps them in spatial joins covering most cases, you may not
> even need to access `sindex` directly.
>
> ``` python
> world_countries.iloc[world_countries.sindex.query(paris, "within")]
> ```

## Spatial join

One of the typical geospatial tasks is a spatial join. Let’s change the
data set again and load boundaries of Spanish cities derived from
@arribas2019building, which proposes a machine learning algorithm to
delineate city boundaries from building footprints.

You can try to figure out which of these cities fall into Catalonia
region and which province they belong to.

First, you need data representing Catalonia. You can download it from
[ICGC](https://www.icgc.cat/en/Downloads/Vector-maps/Administrative-boundaries)
website or use the `catalonia.gpkg` saved in this repository.

In [ ]:
catalonia = gpd.read_file(
    "https://martinfleischmann.net/sds/geographic_data/data/catalonia.gpkg",
)
catalonia.head()

> **Alternative**
>
> Instead of reading the file directly off the web, it is possible to
> download it manually, store it on your computer, and read it locally.
> To do that, you can follow these steps:
>
> 1.  Download the file by right-clicking on [this
>     link](https://martinfleischmann.net/sds/geographic_data/data/catalonia.gpkg)
>     and saving the file
> 2.  Place the file in the same folder as the notebook where you intend
>     to read it
> 3.  Replace the code in the cell above with:
>
> ``` python
> catalonia = gpd.read_file("catalonia.gpkg")
> ```

Then, you can load the boundaries of Spanish cities from the data
repository linked to the paper.

In [ ]:
cities = gpd.read_file("https://ndownloader.figshare.com/files/20232174")
cities.head()

A quick exploration to better understand what you have just opened might
help.

In [ ]:
m = catalonia.explore()
cities.explore(m=m, color="red")

You should check the CRS, because for spatial join, you need to be sure
that both `GeoDataFrames` are using the same (but don’t worry,
`geopandas` would warn you in case of a CRS mismatch).

In [ ]:
catalonia.crs.equals(cities.crs)

Since these two differ, you can re-project the geometries of `catalonia`
to the CRS of `cities`.

In [ ]:
catalonia = catalonia.to_crs(cities.crs)
catalonia.crs.equals(cities.crs)

Now, you can do the spatial join using the `.sjoin()` method. That uses
the `intersects` predicate by default, but you can use any predicates
you used above (plus a couple more).

In [49]:
cities_within_catalonia = cities.sjoin(catalonia)

Let’s see the result.

In [ ]:
print(
    f"Length of cities: {len(cities)}\n"
    f"Length of cities_within_catalonia: {len(cities_within_catalonia)}"
)

You can check that the resulting subset of cities has additional
columns.

In [ ]:
cities_within_catalonia.head(2)

You can now plot cities based on the province they belong to, captured
by the `"NOMPROV"` column.

In [ ]:
cities_within_catalonia.explore(
    "NOMPROV",
    tiles="CartoDB Positron",
)

Since `GeoDataFrames` are still based on `pandas.ataFrames`, we can
readily use `pandas` functionality, like `.groupby()`, on the result.

In [ ]:
cities_within_catalonia.groupby("NOMPROV")["n_buildings"].sum()

## Overlay

Sometimes, you may want to create new geometries based on the spatial
relationships between existing geometries. These are called *overlay*
operations.

Let’s assume that you are interested in areas that are 10 kilometres
around a centroid of each city.

In [ ]:
buffered_centroids = cities_within_catalonia.centroid.buffer(10_000)
buffered_centroids.explore()

With that, you can ask for an intersection between the buffer and city
boundaries.

<figure>
<img
src="https://geopandas.org/en/stable/_images/binary_geo-intersection.svg"
alt="A diagram illustrating the instersection overlay operation." />
<figcaption aria-hidden="true">A diagram illustrating the
<code>instersection</code> overlay operation.</figcaption>
</figure>

In [ ]:
cities_within_catalonia.overlay(
    buffered_centroids.to_frame(), how="intersection"
).explore()

Or you may want to do the union of the two to get the areas that are
either within the city boundary or within 10 km of a city centroid.

<figure>
<img src="https://geopandas.org/en/stable/_images/binary_geo-union.svg"
alt="A diagram illustrating the union overlay operation." />
<figcaption aria-hidden="true">A diagram illustrating the
<code>union</code> overlay operation.</figcaption>
</figure>

In [ ]:
cities_within_catalonia.overlay(
    buffered_centroids.to_frame(), how="union"
).explore()

Or find all parts of city boundaries that are further away than 10km
from each centroid.

<figure>
<img
src="https://geopandas.org/en/stable/_images/binary_geo-difference.svg"
alt="A diagram illustrating the difference overlay operation." />
<figcaption aria-hidden="true">A diagram illustrating the
<code>difference</code> overlay operation.</figcaption>
</figure>

In [ ]:
cities_within_catalonia.overlay(
    buffered_centroids.to_frame(), how="difference"
).explore()

Finally, you may ask which parts of areas are within 10 km or a boundary
but not in both.

<figure>
<img
src="https://geopandas.org/en/stable/_images/binary_geo-symm_diff.svg"
alt="A diagram illustrating symmetric difference overlay operation." />
<figcaption aria-hidden="true">A diagram illustrating
<code>symmetric difference</code> overlay operation.</figcaption>
</figure>

In [ ]:
cities_within_catalonia.overlay(
    buffered_centroids.to_frame(), how="symmetric_difference"
).explore()

If you remember the spatial join between the provinces of Catalonia and
Spanish cities, you may remember that some geometries in the result were
duplicated because those cities intersect with more than one province.
In that case, `.overlay()` may be a better solution. See if you can find
the difference.

In [ ]:
catalonia.overlay(cities).explore(
    "NOMPROV",
    tiles="CartoDB Positron",
)

> **Additional reading**
>
> -   Check the similar introduction by @darribas_gds_course in his part
>     [*Spatial
>     data*](https://darribas.org/gds_course/content/bC/lab_C.html) to
>     learn more about static plots and how to get a background map with
>     those.
> -   Have a look at the chapter [*Choropleth
>     Mapping*](https://geographicdata.science/book/notebooks/05_choropleth.html)
>     explaining how to get choropleth maps from the Geographic Data
>     Science with Python by @rey2023geographic.

## Acknowledgements

Delineation of Spanish cities is used following the *A Course on
Geographic Data Science* by @darribas_gds_course, licensed under
CC-BY-SA 4.0. The section on basic overlay predicates is adapted from
the *Introduction to geospatial data analysis with GeoPandas and the
PyData stack* by [Joris van den
Bossche](https://github.com/jorisvandenbossche/geopandas-tutorial).
Illustrations are taken from the [GeoPandas
documentation](https://geopandas.org), drawn by Martin Fleischmann.